In [3]:
api_key = "hh8DW29tdDmshbghN71NJPI0jgFzp1Ay5MzjsniIz9a8Piq7xp"

### Making the API request for all cards

In [4]:
import requests
headers = {'X-Mashape-Key': api_key}

In [5]:
# cardname = "Leeroy"

In [6]:
# endpoint_one_card = f"https://omgvamp-hearthstone-v1.p.mashape.com/cards/search/{cardname}"
# endpoint_allcards = "https://omgvamp-hearthstone-v1.p.mashape.com/cards"

def get_cards_by_type(card_type, collectible=1, cost=None, durability=None, health=None, key=api_key):
    endpoint_by_type = f'https://omgvamp-hearthstone-v1.p.mashape.com/cards/types/{card_type}'
    payload = {'collectible': collectible, 'cost': cost, 'durability': durability, 'health': health}
    r = requests.get(endpoint_by_type, params=payload, headers=headers)
    return r.json()

In [7]:
# r = requests.get(endpoint_allcards, headers=headers)

In [8]:
# r.json()

In [1]:
import json
# with open('hs_data.json', 'w') as outfile:
#     json.dump(r.json(), outfile)

### Using the offline JSON

In [2]:
import pandas as pd

In [33]:
# Reading the json as a dict
with open('hs_data.json') as json_data:
    data = json.load(json_data)

In [37]:
everything = [single_card for cardset in data.values() for single_card in cardset]

In [39]:
collectibles = [single_card for single_card in everything 
                if 'collectible' in single_card 
                and single_card['collectible']]

In [41]:
non_heroes = [single_card for single_card in collectibles
              if single_card['type'] != 'Hero']

In [53]:
all_types = set([card['type'] for card in non_heroes])
all_types

{'Minion', 'Spell', 'Weapon'}

### Otherwise, first separate the cards by type. We can make API calls for that.

In [11]:
collectible_spells = get_cards_by_type('Spell')
collectible_weapons = get_cards_by_type('Weapon')
collectible_minions = get_cards_by_type('Minion')

In [12]:
minions = collectible_minions
weapons = collectible_weapons
spells = collectible_spells

In [13]:
# for cardtype, collection in zip(['minions', 'spells', 'weapons'], [collectible_minions, collectible_spells, collectible_weapons]):
#     with open(f'{cardtype}.json', 'w') as outfile:
#         json.dump(collection, outfile)

In [71]:
# Reading the json as a dict
with open('data/minions.json') as json_data:
    minions = json.load(json_data)
    
with open('data/spells.json') as json_data:
    spells = json.load(json_data)
    
with open('data/weapons.json') as json_data:
    weapons = json.load(json_data)

### Let's now separate the titles, flavors, texts, mechanics, costs and stats.
#### Each card type has different attributes and design logic behind them, so we want to make educated splits. Moreover, not all fields are of interest for us for now. 

In [72]:
spell_attributes = {"name", "cardId", "cost", "img", "playerClass", "rarity", "text", "flavor", "mechanics"}
# spell_optional_attributes = {}

minion_attributes = {"name", "cardId", "cost", "health", "attack", "img", "playerClass", "rarity", "text", "flavor", "mechanics"}
# minion_optional_attributes = {}

weapon_attributes = {"name", "cardId", "cost", "durability", "attack", "img", "playerClass", "rarity", "text", "flavor", "mechanics"}
# weapon_attributes = {}

### For this first generation example we shall funnel all the cards to the above attributes to normalize the data.

In [73]:
def normalize_card(card, attrs):
    
    concise = {a: card[a] if a in card else None for a in attrs}
    
    return concise

In [74]:
spells_concise = [normalize_card(spell_card, spell_attributes) for spell_card in spells]
minions_concise = [normalize_card(minion_card, minion_attributes) for minion_card in minions]
weapons_concise = [normalize_card(weapon_card, weapon_attributes) for weapon_card in weapons]

In [8]:
spells_concise[:2]

[{'cardId': 'CS2_041',
  'cost': 0,
  'flavor': 'I personally prefer some non-ancestral right-the-heck-now healing, but maybe that is just me.',
  'img': 'http://media.services.zam.com/v1/media/byName/hs/cards/enus/CS2_041.png',
  'mechanics': [{'name': 'Taunt'}],
  'name': 'Ancestral Healing',
  'playerClass': 'Shaman',
  'rarity': 'Free',
  'text': 'Restore a minion\\nto full Health and\\ngive it <b>Taunt</b>.'},
 {'cardId': 'CS2_072',
  'cost': 0,
  'flavor': 'It\'s funny how often yelling "Look over there!" gets your opponent to turn around.',
  'img': 'http://media.services.zam.com/v1/media/byName/hs/cards/enus/CS2_072.png',
  'mechanics': None,
  'name': 'Backstab',
  'playerClass': 'Rogue',
  'rarity': 'Free',
  'text': 'Deal $2 damage to an undamaged minion.'}]

In [17]:
# titles = [(card['name'], card['cardId']) for card in non_heroes]
# flavors = [(card['flavor'], card['cardId']) for card in non_heroes]
# texts = [(card['text'], card['cardId']) for card in non_heroes]
# mechanics = [(card['mechanics'], card['cardId']) if 'mechanics' in card else None for card in non_heroes]
# costs = [(card['cost'], card['cardId']) for card in non_heroes]
# stats = [(card['name'], card['cardId']) for card in non_heroes]

### The data is now quite ready to be modelled! We can now access the needed fields directly from the correpsonding cardtype set.

## A small LSTM

In [102]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

### We will first try to generate card flavors, as those resemble normal text the most. For the flavors we can combine all the flavors from all types of cards, as they are mostly just humorous pieces of text and are not directly associated with the type.

In [76]:
def get_all_flavortexts():
    
    weapon_flavors = [card['flavor'] for card in weapons]
    minion_flavors = [card['flavor'] for card in minions]
    spell_flavors = [card['flavor'] for card in spells if 'flavor' in card]
    
    return weapon_flavors + minion_flavors + spell_flavors

In [77]:
all_flavors = get_all_flavortexts()

In [78]:
all_flavors[73]

'Clockwork gnomes are always asking what time it is.'

### Our average sequence length is 67.8 characters. This is handy to know for the LSTM sequence length parameter. Let's round it down for sake of memorability.

In [79]:
average_sequence_length = np.mean([len(list(flavor)) for flavor in all_flavors])
average_sequence_length

67.86418193303854

In [80]:
SEQUENCE_LENGTH = 65

### Our generative model is a character based one, so our input data is a huge list of characters.

In [81]:
all_flavor_chars = [ch for one_sentence in all_flavors for ch in list(one_sentence)]

In [82]:
all_flavor_unique_chars = set(all_flavor_chars)

### We see that there are a lot of unnecessary or unwished characters in the model. We could clean up the model by moderating the char list but for now we let it like this.

In [83]:
print(f'Data length: {len(all_flavor_chars)} characters')
print(f'Vocabulary size: {len(all_flavor_unique_chars)} characters')

Data length: 107429 characters
Vocabulary size: 94 characters


### The unique characters are the features for our model. Let's numerify them to make them ML ready.

In [84]:
ix_to_char = {ix:char for ix, char in enumerate(all_flavor_unique_chars)}
char_to_ix = {char:ix for ix, char in enumerate(all_flavor_unique_chars)}

### LSTM expects input of the shape (batch_size, length_of_sequence, number_features)
#### batch_size: amount of sequences which are fed into the network at one time, just as in a regular feedforward neural network
#### length_of_sequence: the amount of "neural networks", the memory or the amount of steps the network looks at at each step. In our example, we want to predict a character given 65 previous characters.
#### number_features: the length of one featurized element. In the case of images it could be padded standardized vectors of pixels. In case of text it is the length of our vocab, because our input is going to be represented by every char in our vocabulary.


In [85]:
NUMBER_FEATURES = len(all_flavor_unique_chars)

In [86]:
X = np.zeros((int(len(all_flavor_chars)/SEQUENCE_LENGTH), SEQUENCE_LENGTH, NUMBER_FEATURES))
y = np.zeros((int(len(all_flavor_chars)/SEQUENCE_LENGTH), SEQUENCE_LENGTH, NUMBER_FEATURES))

In [87]:
len(all_flavor_chars)

107429

In [88]:
for i in range(0, int(len(all_flavor_chars)/SEQUENCE_LENGTH)):
    X_sequence = all_flavor_chars[i*SEQUENCE_LENGTH:(i+1)*SEQUENCE_LENGTH]  #Get next sequence of length 65 as input.
    X_sequence_ix = [char_to_ix[value] for value in X_sequence]  # Convert the above sequence to the integer mapping.
    # TODO: make this one hot encoding differently: Keras or sklearn or something.
    input_sequence = np.zeros((SEQUENCE_LENGTH, NUMBER_FEATURES))  # Create a skeleton for the input sequence: we create a 2d numpy matrix which has a feature array of 94 
                                                                   # long for each of the 65 characters in sequence. This way we basically one hot encode our sequences. 
    for j in range(SEQUENCE_LENGTH):  # The one hot encoding process: we replace a zero with a one on a position in the input sequence which corresponds with the index of a character in our converted array!
        input_sequence[j][X_sequence_ix[j]] = 1.
    X[i] = input_sequence
    
    #Same for y!
    y_sequence = all_flavor_chars[i*SEQUENCE_LENGTH+1:(i+1)*SEQUENCE_LENGTH+1]
    y_sequence_ix = [char_to_ix[value] for value in y_sequence]
    target_sequence = np.zeros((SEQUENCE_LENGTH, NUMBER_FEATURES))
    for j in range(SEQUENCE_LENGTH):
        target_sequence[j][y_sequence_ix[j]] = 1.
    y[i] = target_sequence

In [89]:
twee = [char_to_ix[value] for value in all_flavor_chars[1*SEQUENCE_LENGTH:(1+1)*SEQUENCE_LENGTH]]
drie = np.zeros((SEQUENCE_LENGTH, NUMBER_FEATURES))

In [90]:
input_sequence[0][twee[0]]

0.0

In [54]:
X_sequence_ix
y_sequence_ix

[30,
 43,
 27,
 87,
 27,
 25,
 56,
 79,
 45,
 27,
 62,
 47,
 56,
 25,
 47,
 87,
 56,
 6,
 78,
 14,
 14,
 20,
 49,
 4,
 78,
 55,
 87,
 56,
 25,
 47,
 87,
 56,
 25,
 48,
 23,
 87,
 56,
 12,
 43,
 3,
 56,
 78,
 27,
 56,
 87,
 74,
 48,
 14,
 56,
 14,
 78,
 45,
 86,
 47,
 56,
 78,
 12,
 25,
 87,
 3,
 56,
 89,
 43,
 45,
 56]

In [91]:

# # prepare the dataset of input to output pairs encoded as integers
# seq_length = 100
# dataX = []
# dataY = []
# n_chars = len(all_flavor_chars)
# for i in range(0, n_chars - SEQUENCE_LENGTH, 1):
#     seq_in = all_flavor_chars[i:i + SEQUENCE_LENGTH]
#     seq_out = all_flavor_chars[i + SEQUENCE_LENGTH]
#     dataX.append([char_to_ix[char] for char in seq_in])
#     dataY.append(char_to_ix[seq_out])
# n_patterns = len(dataX)
# print("Total Patterns: ", n_patterns)

In [92]:
# X = np.reshape(dataX, (n_patterns, SEQUENCE_LENGTH, 1))

In [93]:
# # normalize
# X = X / float(len(all_flavor_unique_chars))
# # one hot encode the output variable
# y = np_utils.to_categorical(dataY)

In [98]:
HIDDEN_DIM = 500
LAYER_NUM = 3

In [112]:
model = Sequential()
model.add(LSTM(HIDDEN_DIM, input_shape=(None, NUMBER_FEATURES), return_sequences=True))
for i in range(LAYER_NUM - 1):
    model.add(LSTM(HIDDEN_DIM, return_sequences=True))
    model.add(Dropout(0.25))
model.add(TimeDistributed(Dense(NUMBER_FEATURES)))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [115]:
def generate_text(model, length):
    ix = [np.random.randint(NUMBER_FEATURES)]
    y_char = [ix_to_char[ix[-1]]]
    X = np.zeros((1, length, NUMBER_FEATURES))
    for i in range(length):
        X[0, i, :][ix[-1]] = 1
        print(ix_to_char[ix[-1]], end="")
        ix = np.argmax(model.predict(X[:, :i+1, :])[0], 1)
    return ('').join(y_char)

In [116]:
nb_epoch = 0
BATCH_SIZE = 50
GENERATE_LENGTH = SEQUENCE_LENGTH
while True:
    print('\n')
    model.fit(X, y, batch_size=BATCH_SIZE, verbose=1, nb_epoch=1)
    nb_epoch += 1
    generate_text(model, GENERATE_LENGTH)
    if nb_epoch % 10 == 0:
        model.save_weights('checkpoint_{}_epoch_{}.hdf5'.format(HIDDEN_DIM, nb_epoch))



Epoch 1/1
  50/1652 [..............................] - ETA: 5s - loss: 3.1706

/Users/markberger/anaconda/lib/python3.6/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


1652/1652 [==============================] - 5s 3ms/step - loss: 3.2074
9    e   a   e  e  e  e  e  e  e  e  e  e  e  e  e  e  e  e  e  e9    e   a   e  e  e  e  e  e  e  e  e  e  e  e  e  e  e  e  e  e 


Epoch 1/1
1652/1652 [==============================] - 5s 3ms/step - loss: 3.1335
p  oe  oe  oe  oe  oe  oe  oe  oe  oe  oe  oe  oe  oe  oe  oe  oep  oe  oe  oe  oe  oe  oe  oe  oe  oe  oe  oe  oe  oe  oe  oe  oe 


Epoch 1/1
1652/1652 [==============================] - 5s 3ms/step - loss: 3.0486
i  on ton  oo  oo  oo  oo  oo  oo  oo  oo  oo  oo  oo  oo  oo  ooi  on ton  oo  oo  oo  oo  oo  oo  oo  oo  oo  oo  oo  oo  oo  oo 


Epoch 1/1
1652/1652 [==============================] - 5s 3ms/step - loss: 2.9553
%  on te te te te te te te te te te te te te te te te te te te te%  on te te te te te te te te te te te te te te te te te te te te 


Epoch 1/1
1652/1652 [==============================] - 5s 3ms/step - loss: 2.8654
l on an an an an an an an an an an an an an an an an an an an an

KeyboardInterrupt: 